# Módulo 13: Modelos  de Regressão
# Exercício 2: Regularização Ridge Lasso e StepWise

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [43]:
# carregando bibliotecas

import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.tree import export_graphviz
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings

%matplotlib inline

In [44]:
# Desativar avisos de FutureWarning específicos

warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

In [45]:
# carregando arquivo e construindo DataFrame

arquivo = pd.read_csv('previsao_de_renda.csv')
df_previsao = pd.DataFrame(arquivo)

In [46]:
# informações sobre os tipos dos dados

df_previsao.dtypes

Unnamed: 0                 int64
data_ref                  object
id_cliente                 int64
sexo                      object
posse_de_veiculo            bool
posse_de_imovel             bool
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                      int64
tempo_emprego            float64
qt_pessoas_residencia    float64
renda                    float64
dtype: object

In [47]:
# informações sobre os dados

df_previsao.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [48]:
# informações sobre o número de linhas e colunas do dataFrame

print(f'O número de "linhas" do DataFrame é:   {df_previsao.shape[0]}')
print(f'O número de "colunas" do DataFrame é:   {df_previsao.shape[1]}')

O número de "linhas" do DataFrame é:   15000
O número de "colunas" do DataFrame é:   15


In [49]:
# verificando dados faltantes

df_previsao.isna().sum()

Unnamed: 0                  0
data_ref                    0
id_cliente                  0
sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64

In [50]:
# observando a porcentagem de dados faltantes

porcentagem_dados_faltantes = (df_previsao.isnull().sum() / len(df_previsao)) * 100
porcentagem_dados_faltantes

Unnamed: 0                0.000000
data_ref                  0.000000
id_cliente                0.000000
sexo                      0.000000
posse_de_veiculo          0.000000
posse_de_imovel           0.000000
qtd_filhos                0.000000
tipo_renda                0.000000
educacao                  0.000000
estado_civil              0.000000
tipo_residencia           0.000000
idade                     0.000000
tempo_emprego            17.153333
qt_pessoas_residencia     0.000000
renda                     0.000000
dtype: float64

In [51]:
# dropar todas as linhas de "tempo_emprego" que tenha NA

df_previsao.dropna(subset=['tempo_emprego'], inplace=True)

In [52]:
# informações sobre o número de linhas e colunas do dataFrame depois de dropar

print(f'O número de "linhas" do DataFrame é:   {df_previsao.shape[0]}')
print(f'O número de "colunas" do DataFrame é:   {df_previsao.shape[1]}')

O número de "linhas" do DataFrame é:   12427
O número de "colunas" do DataFrame é:   15


In [53]:
# deletando colunas dispensáveis

df_previsao = df_previsao.drop('Unnamed: 0', axis=1, errors='ignore')
df_previsao = df_previsao.drop('data_ref', axis=1, errors='ignore')
df_previsao = df_previsao.drop('id_cliente', axis=1, errors='ignore')

In [54]:
# divisão dos dados em y e X

y = df_previsao.renda
X = df_previsao.drop('renda',axis=1)

In [55]:
# separando os dados em TREINO com 75% e TESTE com 25%

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [56]:
# Identificando as variáveis categóricas

colunas_categoricas = X.select_dtypes(include=['object']).columns


In [57]:
# Aplicando a codificação ONE-HOT para as variáveis categóricas

preprocessor = ColumnTransformer(                       # A codificação ONE-HOT (também conhecida como one-of-N 
    transformers=[                                      # ou dummy encoding). É uma técnica usada para 
        ('cat', OneHotEncoder(), colunas_categoricas)   # representar variáveis categóricas como vetores binários.
    ],
    remainder='passthrough'
)

X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

In [58]:
# 2. Regularização usando RIDGE - usando "sklearn.linear_model"

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]           # lista com os alphas a serem testados
resultados_alphas = {}                                # dicionário para guardar os resultados

for alpha in alphas:
    modelo = Ridge(alpha = alpha)
    modelo.fit(X_train_encoded, y_train)
    y_predito = modelo.predict(X_test_encoded)
    r2 = r2_score(y_test, y_predito)
    resultados_alphas[alpha] = r2
    
melhor_alpha = max(resultados_alphas, key = resultados_alphas.get)
melhor_r2 = resultados_alphas[melhor_alpha]


print ('Lista com os valores de Alpha:')
 
for chave, valor in resultados_alphas.items():
    print(f"{chave}, {valor}")
    
print ('----------------------------------------------------------------')
print (f' O melhor Alpha é: {melhor_alpha}')
print (f' O melhor R2 é: {melhor_r2}')

Lista com os valores de Alpha:
0, 0.291616621890886
0.001, 0.2925875472520665
0.005, 0.29258753190059383
0.01, 0.2925875126758868
0.05, 0.29258735749720666
0.1, 0.2925871602300163
----------------------------------------------------------------
 O melhor Alpha é: 0.001
 O melhor R2 é: 0.2925875472520665


In [59]:
# 3. Regularização usando LASSO - usando "sklearn.linear_model"

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]           # lista com os alphas a serem testados
resultados_alphas = {}                                # dicionário para guardar os resultados

for alpha in alphas:
    modelo = Lasso(alpha = alpha)
    modelo.fit(X_train_encoded, y_train)
    y_predito = modelo.predict(X_test_encoded)
    r2 = r2_score(y_test, y_predito)
    resultados_alphas[alpha] = r2
    
melhor_alpha = max(resultados_alphas, key = resultados_alphas.get)
melhor_r2 = resultados_alphas[melhor_alpha]


print ('Lista com os valores de Alpha:')
 
for chave, valor in resultados_alphas.items():
    print(f"{chave}, {valor}")
    
print ('----------------------------------------------------------------')
print (f' O melhor Alpha é: {melhor_alpha}')
print (f' O melhor R2 é: {melhor_r2}')


C:\Users\Cristina Notebook\anaconda3\Lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\Cristina Notebook\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Cristina Notebook\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.968e+11, tolerance: 7.878e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV inste

Lista com os valores de Alpha:
0, 0.2925876228382642
0.001, 0.2925876214391724
0.005, 0.2925876123429738
0.01, 0.2925875930956625
0.05, 0.2925880165036483
0.1, 0.29258507593118355
----------------------------------------------------------------
 O melhor Alpha é: 0.05
 O melhor R2 é: 0.2925880165036483


C:\Users\Cristina Notebook\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.208e+10, tolerance: 7.878e+07
  model = cd_fast.enet_coordinate_descent(


- Qual método chega a um melhor resultado?
- **Resposta:** Os resultados usando os método de regularização RIDGE e LASSO têm resultados semalhantes. O maior **R
2** é do Alpha **0.05** da regularização **LASSO**.

In [60]:
# carregando arquivo e construindo DataFrame

arquivo = pd.read_csv('previsao_de_renda.csv')
df_previsao = pd.DataFrame(arquivo)

In [61]:
# dropar todas as linhas de "tempo_emprego" que tenha NA

df_previsao.dropna(subset=['tempo_emprego'], inplace=True)

In [62]:
# deletando colunas dispensáveis

df_previsao = df_previsao.drop('Unnamed: 0', axis=1, errors='ignore')
df_previsao = df_previsao.drop('data_ref', axis=1, errors='ignore')
df_previsao = df_previsao.drop('id_cliente', axis=1, errors='ignore')

In [63]:
# separando os dados em TREINO com 75% e TESTE com 25%

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [64]:
# Identificando as variáveis categóricas

colunas_categoricas = X.select_dtypes(include=['object']).columns

In [65]:
# Aplicando a codificação ONE-HOT para as variáveis categóricas

preprocessor = ColumnTransformer(                       # A codificação ONE-HOT (também conhecida como one-of-N 
    transformers=[                                      # ou dummy encoding). É uma técnica usada para 
        ('cat', OneHotEncoder(), colunas_categoricas)   # representar variáveis categóricas como vetores binários.
    ],
    remainder='passthrough'
)

X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

In [66]:
# tipos de dados

info_variaveis = pd.DataFrame({
    'Variável': df_previsao.columns,
    'Tipo de Dados': df_previsao.dtypes
})

info_variaveis.head(info_variaveis.shape[0])


,Variável,Tipo de Dados
sexo,sexo,object
posse_de_veiculo,posse_de_veiculo,bool
posse_de_imovel,posse_de_imovel,bool
qtd_filhos,qtd_filhos,int64
tipo_renda,tipo_renda,object
educacao,educacao,object
estado_civil,estado_civil,object
tipo_residencia,tipo_residencia,object
idade,idade,int64
tempo_emprego,tempo_emprego,float64


In [67]:
# valores da variável - SEXO
contagem_sexos = df_previsao['sexo'].value_counts()
print(contagem_sexos)
print("___________________________________________")

# valores da variável - TIPO RENDA
contagem_tipo_renda = df_previsao['tipo_renda'].value_counts()
print(contagem_tipo_renda)
print("___________________________________________")

# valores da variável - EDUCACAO
contagem_educacao 	 = df_previsao['educacao'].value_counts()
print(contagem_educacao)
print("___________________________________________")

# valores da variável - ESTADO CIVIL
contagem_estado_civil = df_previsao['estado_civil'].value_counts()
print(contagem_estado_civil)
print("___________________________________________")

# valores da variável - TIPO DE RESIDENCIA
contagem_tipo_residencia = df_previsao['tipo_residencia'].value_counts()
print(contagem_tipo_residencia)
print("___________________________________________")

sexo
F    7901
M    4526
Name: count, dtype: int64
___________________________________________
tipo_renda
Assalariado         7633
Empresário          3508
Servidor público    1268
Bolsista               9
Pensionista            9
Name: count, dtype: int64
___________________________________________
educacao
Secundário             7045
Superior completo      4695
Superior incompleto     558
Primário                103
Pós graduação            26
Name: count, dtype: int64
___________________________________________
estado_civil
Casado      8897
Solteiro    1543
União        924
Separado     739
Viúvo        324
Name: count, dtype: int64
___________________________________________
tipo_residencia
Casa             11071
Com os pais        674
Governamental      360
Aluguel            183
Estúdio             75
Comunitário         64
Name: count, dtype: int64
___________________________________________


In [68]:
# variáveis dummies

# variável "sexo"
df_previsao['sexo'] = df_previsao['sexo'].map({
    'F': 1.0,
    'M': 0.0
})


# variável "tipo_renda"
df_previsao['tipo_renda'] = df_previsao['tipo_renda'].map({
    'Assalariado': 1.0,
    'Empresário': 2.0,
    'Servidor público': 3.0,
    'Bolsista': 4.0,
    'Pensionista': 5.0
})


# variável "educacao"
df_previsao['educacao'] = df_previsao['educacao'].map({
    'Secundário': 1.0,
    'Superior completo': 2.0,
    'Superior incompleto': 3.0,
    'Primário': 4.0,
    'Pós graduação': 5.0
})


# variável "estado_civil"
df_previsao['estado_civil'] = df_previsao['estado_civil'].map({
    'Casado': 1.0,
    'Solteiro': 2.0,
    'União': 3.0,
    'Separado': 4.0,
    'Viúvo': 5.0
})


# variável "tipo_residencia"
df_previsao['tipo_residencia'] = df_previsao['tipo_residencia'].map({
    'Casa': 1.0,
    'Com os pais': 2.0,
    'Governamental': 3.0,
    'Aluguel': 4.0,
    'Estúdio': 5.0,
    'Comunitário': 6.0
})


# Converter colunas bool para numérico (0 ou 1)
df_previsao['posse_de_veiculo'] = df_previsao['posse_de_veiculo'].astype(int)
df_previsao['posse_de_imovel'] = df_previsao['posse_de_imovel'].astype(int)

     
# Remover linhas com inf ou nan - senão o Stepwise não funciona
df_cleaned = df_previsao.replace([np.inf, -np.inf], np.nan).dropna()


In [69]:
# tipos de dados

info_variaveis = pd.DataFrame({
    'Variável': df_previsao.columns,
    'Tipo de Dados': df_previsao.dtypes
})

info_variaveis.head(info_variaveis.shape[0])

,Variável,Tipo de Dados
sexo,sexo,float64
posse_de_veiculo,posse_de_veiculo,int32
posse_de_imovel,posse_de_imovel,int32
qtd_filhos,qtd_filhos,int64
tipo_renda,tipo_renda,float64
educacao,educacao,float64
estado_civil,estado_civil,float64
tipo_residencia,tipo_residencia,float64
idade,idade,int64
tempo_emprego,tempo_emprego,float64


In [70]:
# verificando os dados

df_cleaned.shape

(12427, 12)

In [71]:
# 4. Rode um modelo stepwise. Avalie o 𝑅2 na base de testes. Qual o melhor resultado?


# Separar variáveis independentes (X) e dependente (y)
X = df_cleaned.drop(columns=['renda'])  
y = df_cleaned['renda']

# separando os dados em TREINO com 75% e TESTE com 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)


def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

# aplicar stepwise na base de testes
variaveis = stepwise_selection(X_test, y_test)

print('resulting features:')
print(variaveis)


Add  tempo_emprego                  with p-value 7.51872e-125
#############
['tempo_emprego']
Add  sexo                           with p-value 1.35913e-105
#############
['tempo_emprego', 'sexo']
Add  tipo_renda                     with p-value 0.000361938
#############
['tempo_emprego', 'sexo', 'tipo_renda']
Add  qtd_filhos                     with p-value 0.000845114
#############
['tempo_emprego', 'sexo', 'tipo_renda', 'qtd_filhos']
Add  idade                          with p-value 0.000294941
#############
['tempo_emprego', 'sexo', 'tipo_renda', 'qtd_filhos', 'idade']
Add  educacao                       with p-value 0.00944773
#############
['tempo_emprego', 'sexo', 'tipo_renda', 'qtd_filhos', 'idade', 'educacao']
Add  posse_de_imovel                with p-value 0.022683
#############
['tempo_emprego', 'sexo', 'tipo_renda', 'qtd_filhos', 'idade', 'educacao', 'posse_de_imovel']
#############
['tempo_emprego', 'sexo', 'tipo_renda', 'qtd_filhos', 'idade', 'educacao', 'posse_de_imovel']

In [72]:
# Ajustar o modelo com as variáveis selecionadas no conjunto de teste

X_test_selected = X_test[variaveis]
X_test_selected = sm.add_constant(X_test_selected)
model_teste = sm.OLS(y_test, X_test_selected).fit()

# Fazer previsões usando o modelo ajustado na base de testes
y_pred_teste = model_teste.predict(X_test_selected)

# Calcular o R^2 na base de testes
r2_teste = r2_score(y_test, y_pred_teste)
print(f"R^2 na Base de Testes: {r2_teste:.4f}")

R^2 na Base de Testes: 0.2961


- Qual método chega a um melhor resultado?
- **Resposta:**
- R2 Ridge: 0.292587
- R2 Lasso: 0.292588
- R2 Stepwise: 0.2961
- O melhor resultado é usando o método **Stepwise** que tem um **R2** Maior.

In [75]:
# 6. Partindo dos modelos que você ajustou, tente melhorar o 𝑅2 na base de testes. Use a criatividade, 
# veja se consegue inserir alguma transformação ou combinação de variáveis


# usar o modelo que foi aplicado o StepWise

reg = smf.ols('np.log(renda) ~ C(tipo_renda) + sexo + posse_de_imovel + idade + tempo_emprego + qtd_filhos + educacao', data = df_cleaned).fit()

reg.summary()


# R2 maior de 3.353

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     678.6
Date:                Sat, 13 Jan 2024   Prob (F-statistic):               0.00
Time:                        10:44:56   Log-Likelihood:                -13607.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12416   BIC:                         2.732e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                7.8696      0.039    204.247      0.000       7.794       7.945
C(tipo_renda)[T.2.0]     0.1595      0.015     10.711      0.000       0.130       0.189
C(tipo_renda)[T.3.0]     0.0714      0.022      3.223      0.001       0.028       0.115
C(tipo_renda)[T.4.0]     0.2504      0.242      1.036      0.300      -0.223       0.724
C(tipo_renda)[T.5.0]    -0.2857      0.241     -1.184      0.237      -0.759       0.187
sexo                    -0.7984      0.014    -58.064      0.000      -0.825      -0.771
posse_de_imovel          0.0856      0.014      6.240      0.000       0.059       0.113
idade                    0.0057      0.001      7.362      0.000       0.004       0.007
tempo_emprego            0.0616      0.001     59.285      0.000       0.060       0.064
qtd_filhos               0.0309      0.009      3.596      0.000       0.014       0.048
educacao                 0.0482      0.010      4.698      0.000       0.028       0.068
==============================================================================
Omnibus:                        0.976   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.614   Jarque-Bera (JB):                0.951
Skew:                           0.020   Prob(JB):                        0.622
Kurtosis:                       3.017   Cond. No.                     1.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.